### Importing Important Modules 

In [8]:
#Enabling interactive secession for the notebook and save in very 20 second.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
%autosave 20

Autosaving every 20 seconds


In [9]:
from __future__ import print_function
import torch                                #Importing Torch module as ml framework desined for reserchers and developer.
import torch.nn as nn                       #NN module have classes and modules to implement and train the neural network.
import torch.nn.functional as F             #Functional provides module like activations,losses etc
import torch.optim as optim                 #Importing optimizer moduke from torch
from torchvision import datasets, transforms#Importing Torch vision for datasets and data transforms.

In [10]:
import matplotlib.pyplot as plt

In [12]:
#importing a module created by me to calculate receptive field.Having copule of bugs but works fine.
from  rf_calc import receptive_field

In [13]:
torch.__version__ #checking Troch version

'1.13.0'

In [14]:
#Checking for torch GPU support
print(f"Is cuda GPU avalable: {torch.cuda.is_available()}")

Is cuda GPU avalable: True


In [19]:
use_cuda = torch.cuda.is_available() #chjecking if cuda is available or not
device = torch.device("cuda" if use_cuda else "cpu") #if gpu is available then device = cuda else cpu
torch.manual_seed(1)
batch_size = 128
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [21]:
### Defining train loader and test loader 

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data', train=True, download=True,  #downloading the data at /data folder,its for trainning
                    transform=transforms.Compose([       #data transformation includes converting to tensor and normalizing with the mean and std of the dataset
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

### Trainning Function

from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()   #model set to trainning
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar): # Iterating through data and target
        data, target = data.to(device), target.to(device)  #pushing data and target to gpu
        optimizer.zero_grad() # making all the gradients to zero
        output= model(data) # predicting op
        type(output)
        loss = F.nll_loss(output, target) # calculating loss
        loss.backward()  # This is backpropagation in action.
        optimizer.step() # Updating old weights with new 
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

### Testing Function

def test(model, device, test_loader):
    model.eval()  #Model set to evaluation mode.
    test_loss = 0  
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output= model(data)   #predicting the test data
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

###  Model Class defination

In [22]:
# Define the model
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=1, padding=0,bias=True)  #op = 26 * 26
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=1, padding=0,bias=True)  #op = 24 * 24
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3,3), stride=1, padding=0,bias=True)  #op = 22 * 22
        
        self.mp1 = nn.MaxPool2d(kernel_size=2,stride=2) #op = 11 * 11
        
        self.conv4 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3,3), stride=1, padding=1,bias=True)  #op = 11 * 11
        self.conv5 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3), stride=1, padding=0,bias=True)  #op = 9 * 9
        self.conv6 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3), stride=1, padding=0,bias=True)  #op = 7 * 7
        
        self.mp2 = nn.MaxPool2d(kernel_size=2,stride=2) #op = 4 * 4
        
        self.conv7 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), stride=1, padding=1,bias=True)  #op = 4 * 4
        self.conv8 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3,3), stride=1, padding=1,bias=True)  #op = 2 * 2
        self.conv9 = nn.Conv2d(in_channels=256, out_channels=10, kernel_size=(3,3), stride=1, padding=0,bias=True)  #op = 1 * 1
        

    def forward(self, x):
        x = F.relu(self.conv3(F.relu(self.conv2(F.relu(self.conv1(x))))))
        x = self.mp1(x)
        
        x = F.relu(self.conv6(F.relu(self.conv5(F.relu(self.conv4(x))))))
        x = self.mp2(x)
        
        x = self.conv9(F.relu(self.conv8(F.relu(self.conv7(x)))))
        x = x.view(-1,10)
        return F.log_softmax(x,-1)

In [23]:
class Net1(nn.Module):       #created a model class as Net  inhereate properties from nn.Modules
    def __init__(self):     #defining Init function.
        super(Net1, self).__init__()
        self.maxpool = nn.MaxPool2d(kernel_size = (2,2),stride=2)
        self.conv = nn.Conv2d( in_channels=1,out_channels=1,kernel_size=(3,3),stride=1,padding= 1, bias=False, padding_mode = 'replicate')
    
    def forward(self, x):  #forward functions
        x = self.conv(self.maxpool(x))
        return x                          #Final log softmax layer 

In [24]:
from torchsummary import summary   #importing torchsummary.
model = Net().to(device) #creating model and sending it to "CPU/CUDA"
summary(model, input_size=(1, 28, 28))  #Printing the model summary details 

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
            Conv2d-2           [-1, 16, 24, 24]           1,168
            Conv2d-3           [-1, 16, 22, 22]           2,320
         MaxPool2d-4           [-1, 16, 11, 11]               0
            Conv2d-5           [-1, 32, 11, 11]           4,640
            Conv2d-6             [-1, 64, 9, 9]          18,496
            Conv2d-7             [-1, 64, 7, 7]          36,928
         MaxPool2d-8             [-1, 64, 3, 3]               0
            Conv2d-9            [-1, 128, 3, 3]          73,856
           Conv2d-10            [-1, 256, 3, 3]         295,168
           Conv2d-11             [-1, 10, 1, 1]          23,050
Total params: 455,706
Trainable params: 455,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/

### Let's calculate the Receptive field

In [25]:
rf_df = receptive_field(model_obj=model,input_image=28)

=======================================Reciptive Field Calculator========================================
|    | Kernel_size   | Padding   |   Stride | Input_Img_size   | Output_Img_size   | Receptive_field   |
|---:|:--------------|:----------|---------:|:-----------------|:------------------|:------------------|
|  0 | 3*3           | NO        |        1 | 28*28            | 26*26             | 3*3               |
|  1 | 3*3           | NO        |        1 | 26*26            | 24*24             | 5*5               |
|  2 | 3*3           | NO        |        1 | 24*24            | 22*22             | 7*7               |
|  3 | 2*2           | NO        |        2 | 22*22            | 11*11             | 8*8               |
|  4 | 3*3           | 1         |        1 | 11*11            | 11*11             | 12*12             |
|  5 | 3*3           | NO        |        1 | 11*11            | 9*9               | 16*16             |
|  6 | 3*3           | NO        |        1 | 9*9     

In [26]:
i = 1
print (f"Model Reduced the parameter {i} times the base model")
model = Net().to(device) #creating model and sending it to "CPU/CUDA"
summary(model, input_size=(1, 28, 28))  #Printing the model summary details 
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # setting the optimizer

for epoch in range(1, 2):    # running is for 1 epoch
    train(model, device, train_loader, optimizer, epoch)  # Running the Train Function
    test(model, device, test_loader)    # running the test Function

Model Reduced the parameter 1 times the base model
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
            Conv2d-2           [-1, 16, 24, 24]           1,168
            Conv2d-3           [-1, 16, 22, 22]           2,320
         MaxPool2d-4           [-1, 16, 11, 11]               0
            Conv2d-5           [-1, 32, 11, 11]           4,640
            Conv2d-6             [-1, 64, 9, 9]          18,496
            Conv2d-7             [-1, 64, 7, 7]          36,928
         MaxPool2d-8             [-1, 64, 3, 3]               0
            Conv2d-9            [-1, 128, 3, 3]          73,856
           Conv2d-10            [-1, 256, 3, 3]         295,168
           Conv2d-11             [-1, 10, 1, 1]          23,050
Total params: 455,706
Trainable params: 455,706
Non-trainable params: 0
--------------------------------------------

loss=0.19180721044540405 batch_id=468: 100%|██| 469/469 [00:09<00:00, 51.90it/s]



Test set: Average loss: 0.1545, Accuracy: 9510/10000 (95.1000%)



### Let's Take one image and send it to conv and see what's happening 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for batch_idx, (data, target) in enumerate(train_loader): # Iterating through data and target
    data, target = data.to(device), target.to(device)  #pushing data and target to gpu
    break

In [ ]:
_ = plt.imshow(data[0].reshape(28,28,-1).to('cpu'))

In [ ]:
target[0]

In [ ]:
output_val,x1,x2,x3,x4,x5 = model(data[0])

In [ ]:
output_val

In [ ]:
x1.shape
x2.shape
x3.shape
x4.shape
x5.shape

In [ ]:
x5

### Difference between with relu and without relu

##### With Relu

In [ ]:
F.relu(x5)

In [ ]:
F.log_softmax(F.relu(x5),-1)  #Because Relu set the last layer's negative information to zero, the majority of values are identical.

In [ ]:
F.log_softmax(F.relu(x5),-1).squeeze()

In [ ]:
#loss
l = F.nll_loss(F.log_softmax(F.relu(x5),-1).squeeze() , target[0])

In [ ]:
l

##### Without Relu

In [ ]:
x5

In [ ]:
F.log_softmax(x5,-1)  #Because Relu set the last layer's negative information to zero, the majority of values are identical.

In [ ]:
F.log_softmax(x5,-1).squeeze()

In [ ]:
#loss
F.nll_loss(F.log_softmax(x5,-1).squeeze() , target[0])

### Let's Visualize some kernels we trainned

In [ ]:
print_layer(model.conv7.weight.detach().clone())

In [ ]:
print_layer(model.conv6.weight.detach().clone())

In [ ]:
print_layer(model.conv5.weight.detach().clone())

In [ ]:
print_layer(model.conv4.weight.detach().clone())

In [ ]:
print_layer(model.conv3.weight.detach().clone())

In [ ]:
print_layer(model.conv2.weight.detach().clone())

In [ ]:
print_layer(model.conv1.weight.detach().clone())